# Working with Tropospheric Emissions: Monitoring of Pollution (TEMPO)'s Image Service in ArcGIS

## Export Image

In [48]:
from arcgis.gis import GIS
import arcgis
from arcgis.mapping import WebMap
from arcgis.raster import ImageryLayer
from arcgis.raster.functions import colormap, RFT
from datetime import datetime
from arcgis.features import FeatureLayer
from arcgis.mapping import MapImageLayer
from ipywidgets import IntSlider, VBox, Output
from IPython.display import display
import time
import json

def convert_to_milliseconds(date_time_str):
    """Converts a date-time string in 'YYYY-MM-DD HH:MM:SS' format to milliseconds since epoch."""
    dt = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
    milliseconds_since_epoch = int(dt.timestamp() * 1000)
    return milliseconds_since_epoch


# Connect to GIS
gis = GIS()
date_time = 1720617417000 #EST time 
bbox = [-125.0, 22, -66, 51]



# Provide a rendering rule for the color ramp, with a preset ColorRamp from ArcGIS
rendering_rule = {
    "rasterFunctionArguments": {
        "colorRamp": { #custom ColorRamp
            "type": "multipart",
            "colorRamps": [ 
                {
                    "type": "algorithmic",
                    "fromColor": [0, 0, 255, 255],
                    "toColor": [0, 255, 255, 255],
                    "algorithm": "esriCIELabAlgorithm"
                },
                {
                    "type": "algorithmic",
                    "fromColor": [0, 255, 255, 255],
                    "toColor": [255, 255, 0, 255],
                    "algorithm": "esriCIELabAlgorithm"
                },
                {
                    "type": "algorithmic",
                    "fromColor": [255, 255, 0, 255],
                    "toColor": [255, 0, 0, 255],
                    "algorithm": "esriCIELabAlgorithm"
                }
            ]
        },
        "Raster": {
            "rasterFunctionArguments": {
                "StretchType": 5,
                "Statistics": [[0, 30000000000000000, 910863682171422.1, 9474291611234248]], # min value is 0, max value is 3e+16
                "DRA": False,
                "UseGamma": False,
                "Gamma": [1],
                "ComputeGamma": True,
                "Min": 0,
                "Max": 255
            },
            "rasterFunction": "Stretch",
            "outputPixelType": "U64", # must coincide with parameter's pixel type
            "variableName": "Raster"
        }
    },
    "rasterFunction": "Colormap",
    "variableName": "Raster"
}

renderer = {
    "rasterFunction": "Colormap",
    "rasterFunctionArguments": {
        "colorRamp": {
            "type": "multipart",
            "colorRamps": [
                {
                    "type": "algorithmic",
                    "fromColor": [0, 0, 255, 255],
                    "toColor": [0, 255, 255, 255],
                    "algorithm": "esriCIELabAlgorithm"
                },
                {
                    "type": "algorithmic",
                    "fromColor": [0, 255, 255, 255],
                    "toColor": [255, 255, 0, 255],
                    "algorithm": "esriCIELabAlgorithm"
                },
                {
                    "type": "algorithmic",
                    "fromColor": [255, 255, 0, 255],
                    "toColor": [255, 0, 0, 255],
                    "algorithm": "esriCIELabAlgorithm"
                }
            ]
        },
        "StretchType": 5,
        "Statistics": [[0, 30000000000000000, 910863682171422.1, 9474291611234248]],
        "DRA": False,
        "UseGamma": False,
        "Gamma": [1],
        "ComputeGamma": True,
        "Min": 0,
        "Max": 255
    },
    "variableName": "Raster"
}

rendering_rule_json = json.dumps(rendering_rule)



# TEMPO Image Server
image_service_url = "https://gis.earthdata.nasa.gov/image/rest/services/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN_BETA/ImageServer/"

# Load the imagery service
imagery_layer = ImageryLayer(image_service_url)


# Initialize the map centered on the bounding box
center_lat = (bbox[1] + bbox[3]) / 2
center_lon = (bbox[0] + bbox[2]) / 2
map_widget = gis.map(location=[center_lat, center_lon], zoomlevel=4)


# Display the map
map_widget.add_layer(imagery_layer, {'opacity':0.5, 'renderer': renderer})
map_widget.time_slider = True
map_widget.set_time_extent(start_time=datetime(2024, 7, 10), end_time=datetime(2024, 7, 11), unit='minutes')

map_widget



MapView(layout=Layout(height='400px', width='100%'), time_slider=True)

In [50]:
print(arcgis.__version__)

2.3.1
